In [1]:
import os
os.chdir('..')

In [35]:
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import models
import scoring
from data import pronostia
import gc 

MODELS = {
    'mscnn_fen': ('900c195954617834ff774c1b8e154ac81a87c707', 'net_mscnn_pronostia_fold', 110),
    'rnn_fen': ('0f80bc3117fb7b25b8438a4e6fbee62784005748', 'net_rnn_pronostia_fold', 122)
}



_dir = 'rnn_fen'
code = '0f80bc3117fb7b25b8438a4e6fbee62784005748'
file = 'net_rnn_pronostia_fold'
window = 122

In [36]:
results = []


for fold in range(5):

    fen = tf.keras.models.load_model(f'../results/nets/{_dir}/{code}/{file}_{fold}.h5',
                                    custom_objects={'LeakyReLU': tf.keras.layers.LeakyReLU,
                                                    'NASAScore': scoring.NASAScore,
                                                    'PHM21Score': scoring.PHM21Score,
                                                    'SelfAttention': models.SelfAttention})

    test_bearings = [i for i in range(1, 8) if i not in pronostia.FOLD_TRAIN_BEARINGS[fold]]
    X_train, X =  pronostia.load_data(return_test=False, train_bearings=pronostia.FOLD_TRAIN_BEARINGS[fold])
    del X_train



    for bearing, condition in X[['Bearing', 'Condition']].drop_duplicates().values:

        XB = X[(X.Bearing==bearing) & (X.Condition == condition)][['V_acc', 'H_acc']].values
        RUL = X[(X.Bearing==bearing) & (X.Condition == condition)].RUL.values
        y_max, y_min = RUL.max(), RUL.min()

        XB = [XB[i:i+window].T for i in range(0, RUL.shape[0]-window, window)]
        XB = np.expand_dims(np.array(XB), axis=-1)
        YB = [RUL[i+window] / y_max for i in range(0, RUL.shape[0] - window, window)]
        t =  [RUL[i+window] for i in range(0, RUL.shape[0] - window, window)][::-1]

        YP = fen.predict(XB, batch_size=256, verbose=1)
        
        R = pd.DataFrame({'t': t, 'RUL': YB, 'pred': YP[:,0]}).groupby('t').mean().reset_index()

        results.append({
            'fold': fold,
            'bearing': bearing,
            'condition': condition,
            'rul': R.RUL,
            'pred': R.pred,
            't': R.t
        })
        
        print(bearing, condition, max(t))
        
        
    del X
    del fen
    gc.collect()
    
    

[1 2 3 4 5 6 7]
72/72 [==============================] - 0s 2ms/step
2 1 8709
66/66 [==============================] - 0s 2ms/step
2 2 7969
135/135 [==============================] - 0s 2ms/step
2 3 16369
94/94 [==============================] - 0s 2ms/step
4 1 11728
189/189 [==============================] - 0s 2ms/step
5 1 24629
51/51 [==============================] - 0s 2ms/step
4 2 7509
165/165 [==============================] - 0s 2ms/step
5 2 23109
[1 2 3 4 5 6 7]
230/230 [==============================] - 0s 2ms/step
1 1 28029
75/75 [==============================] - 0s 2ms/step
1 2 9109
43/43 [==============================] - 0s 2ms/step
1 3 5149
148/148 [==============================] - 0s 2ms/step
3 1 23749
189/189 [==============================] - 0s 2ms/step
6 1 24479
99/99 [==============================] - 0s 2ms/step
3 2 19549
47/47 [==============================] - 0s 2ms/step
6 2 7009
29/29 [==============================] - 0s 2ms/step
3 3 4339
[1 2 3 4 5 6 7]
72

In [37]:
import pickle as pk
pk.dump(results, open(f'../results/predictions/{_dir}.pk', 'wb'))